Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
# pickle_file = 'notMNIST.pickle'
pickle_file = 'notMNIST_sanitized.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (9633, 28, 28), (9633,))
('Test set', (9604, 28, 28), (9604,))


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    """
    usually we see (in Theano) (batch_sz, channel, height, width), but here it is
    (batch_sz, height(?), width(?), channel)
    """
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28, 1), (200000, 10))
('Validation set', (9633, 28, 28, 1), (9633, 10))
('Test set', (9604, 28, 28, 1), (9604, 10))


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], 
                                                     stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        """
        API: conv2d(input, filter, strides, padding, use_cudnn_on_gpu, name)
        https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#conv2d
        -> expects input tensor of shape `[batch, in_height, in_width, in_channels]`
            -> note that we don't need to specify those sizes - just hand 4d tensor
            with the right values in the right indices
        -> expects filter/kernel of shape `[filter_height, filter_width, in_channels, 
        out_channels]`
            -> note that we don't need to specify the image h & w here, just filter and
            in and out depth
            -> must have strides[0] = stride[3] = 1; for same horiz. and vert. strides, 
            `strides=[1,stride,stride,1]`
            -> so, here, instead of pooling we are using stride=2
        -> padding="SAME" (vs. "VALID") means we go to the edge, so with stride 2, 
        each convolution should divide the image size in half on each axis
        """
        # output shape of first conv. should be [batch, in_h//2, in_w//2, depth]
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        # apply `relu` to tensor, broadcast biases over first three axes? biases vector
        # is of size (depth,) - final shape will not change
        hidden = tf.nn.relu(conv + layer1_biases)
        # output shape of second conv. should be [batch, in_h//4, in_w//4, depth]
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        # apply `relu` to tensor, broadcast biases over first three axes? biases vector
        # is of size (depth,) - final shape will not change
        hidden = tf.nn.relu(conv + layer2_biases)
        #
        # where did `batch` go?...
        #
        # now, after passing through 2 conv and hidden as 2d, we need to think about what
        # the _shape_ of the output data is and match the next layer of weights against
        # that - we've done two convolutons of stride 2, so divide h&w by 4, and we've
        # mapped to `depth` features, so multiply by that -> these are one dimension of
        # the weights matrix; the other is the number of hidden neurons... where is `batch`?
        shape = hidden.get_shape().as_list()
        # we reshape the `hidden` output to be [batch, imgsz//4 * imgsz//4 * depth]
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        # next we matmul against weights matrix with shp [imgsz//4 * imgsz//4 * depth, num_hidden]
        # output is matrix of size [batch, num_hidden]
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        # finally matmul against weights of shp [num_hidden, num_labels]
        return tf.matmul(hidden, layer4_weights) + layer4_biases
        # so the final answer has shape [batch, num_labels]
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.544693
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 50: 1.928339
Minibatch accuracy: 43.8%
Validation accuracy: 42.0%
Minibatch loss at step 100: 1.046906
Minibatch accuracy: 75.0%
Validation accuracy: 62.5%
Minibatch loss at step 150: 0.763929
Minibatch accuracy: 81.2%
Validation accuracy: 64.6%
Minibatch loss at step 200: 1.154092
Minibatch accuracy: 75.0%
Validation accuracy: 72.9%
Minibatch loss at step 250: 0.459403
Minibatch accuracy: 87.5%
Validation accuracy: 74.2%
Minibatch loss at step 300: 0.311103
Minibatch accuracy: 93.8%
Validation accuracy: 74.5%
Minibatch loss at step 350: 0.692385
Minibatch accuracy: 75.0%
Validation accuracy: 74.3%
Minibatch loss at step 400: 0.850487
Minibatch accuracy: 87.5%
Validation accuracy: 76.6%
Minibatch loss at step 450: 0.643262
Minibatch accuracy: 62.5%
Validation accuracy: 76.2%
Minibatch loss at step 500: 0.170487
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], 
                                                     stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        """
        API: conv2d(input, filter, strides, padding, use_cudnn_on_gpu, name=None)
        https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#conv2d
        API: max_pool(value, ksize, strides, padding, name=None)
        https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#max_pool
        -> generally will want ksize like `[1, 2, 2, 1]` for 2x2 pooling - in this
        case, also want the kstrides to be `[1, 2, 2, 1]` so we take non-overlapping
        pooling regions, etc.
        """
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pooled + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pooled + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.942671
Minibatch accuracy: 18.8%
Validation accuracy: 10.3%
Minibatch loss at step 50: 1.918615
Minibatch accuracy: 43.8%
Validation accuracy: 42.3%
Minibatch loss at step 100: 1.156041
Minibatch accuracy: 62.5%
Validation accuracy: 65.7%
Minibatch loss at step 150: 0.652393
Minibatch accuracy: 87.5%
Validation accuracy: 67.6%
Minibatch loss at step 200: 0.852136
Minibatch accuracy: 81.2%
Validation accuracy: 71.8%
Minibatch loss at step 250: 0.666454
Minibatch accuracy: 75.0%
Validation accuracy: 73.2%
Minibatch loss at step 300: 0.396112
Minibatch accuracy: 87.5%
Validation accuracy: 73.9%
Minibatch loss at step 350: 0.777339
Minibatch accuracy: 75.0%
Validation accuracy: 75.7%
Minibatch loss at step 400: 0.772269
Minibatch accuracy: 87.5%
Validation accuracy: 76.3%
Minibatch loss at step 450: 0.564912
Minibatch accuracy: 87.5%
Validation accuracy: 75.5%
Minibatch loss at step 500: 0.189607
Minibatch accuracy: 93.8%
Validation accuracy: 78.8%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [38]:
batch_size = 16
patch_size = 5
depth = 24
num_hidden = 64
l2_w1par = 0.005
l2_w2par = 0.005
l2_w3par = 0.005
l2_w4par = 0.005

dropout_keep_prob1 = 0.75
dropout_keep_prob2 = 0.75

base_learning_rate = 0.025
decay_rate = 0.95
decay_steps = 200

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], 
                                                     stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        """
        API: conv2d(input, filter, strides, padding, use_cudnn_on_gpu, name=None)
        https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#conv2d
        API: max_pool(value, ksize, strides, padding, name=None)
        https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#max_pool
        -> generally will want ksize like `[1, 2, 2, 1]` for 2x2 pooling - in this
        case, also want the kstrides to be `[1, 2, 2, 1]` so we take non-overlapping
        pooling regions, etc.
        """
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(pooled + layer1_biases), keep_prob=dropout_keep_prob1)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.dropout(tf.nn.relu(pooled + layer2_biases), keep_prob=dropout_keep_prob2)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + 
                          l2_w1par * tf.nn.l2_loss(layer1_weights) + 
                          l2_w2par * tf.nn.l2_loss(layer2_weights) + 
                          l2_w3par * tf.nn.l2_loss(layer3_weights) + 
                          l2_w4par * tf.nn.l2_loss(layer4_weights))    

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(base_learning_rate,
                                               step,
                                               decay_steps=decay_steps,
                                               decay_rate=decay_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
##    optimizer = tf.train.GradientDescentOptimizer(base_learning_rate).minimize(loss)


    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [39]:
num_steps = 2001


with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.404591
Minibatch accuracy: 6.2%
Validation accuracy: 10.3%
Minibatch loss at step 50: 4.217673
Minibatch accuracy: 0.0%
Validation accuracy: 14.3%
Minibatch loss at step 100: 4.047334
Minibatch accuracy: 25.0%
Validation accuracy: 22.3%
Minibatch loss at step 150: 3.319253
Minibatch accuracy: 50.0%
Validation accuracy: 40.9%
Minibatch loss at step 200: 2.894224
Minibatch accuracy: 68.8%
Validation accuracy: 49.5%
Minibatch loss at step 250: 2.977365
Minibatch accuracy: 68.8%
Validation accuracy: 56.9%
Minibatch loss at step 300: 2.595073
Minibatch accuracy: 75.0%
Validation accuracy: 63.7%
Minibatch loss at step 350: 2.612658
Minibatch accuracy: 62.5%
Validation accuracy: 65.6%
Minibatch loss at step 400: 2.515273
Minibatch accuracy: 75.0%
Validation accuracy: 67.5%
Minibatch loss at step 450: 2.433262
Minibatch accuracy: 75.0%
Validation accuracy: 68.4%
Minibatch loss at step 500: 2.029131
Minibatch accuracy: 87.5%
Validation accuracy: 69.9%
Min